In [29]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt

#%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [30]:
# Training data is 100 points in [0,1] inclusive regularly spaced
Theta1 = np.linspace(-2,2,10)
Theta2 = np.linspace(-2,2,10)
mesh = np.array(np.meshgrid(Theta1, Theta2))
x = torch.tensor(np.linspace(-2,2,5))

train_T = torch.tensor(mesh.T.reshape(-1, 2))

noise_mean = 0
noise_std = 0.1**2
Theta_True = torch.tensor([1,-1])


# True function is y=T1*x + T2*x^2 + x^3 with Gaussian noise


train_y = torch.tensor(np.zeros([len(train_T),len(x)]))
#Need to fix this to allow all xs to be considered
for i in range(len(train_T)):
    for j in range(len(train_T)):
        for k in range(len(x)):
            y_true =  Theta_True[0]*x[k] + Theta_True[1]*x[k]**2 +x[k]**3 + torch.randn(x[k].size()) * noise_std
            y_exp = train_T[i,0]*x[k] + train_T[j,1]*x[k]**2 +x[k]**3 + torch.randn(x[k].size()) * noise_std
            train_y[i,k] = (y_true - y_exp)**2
            
print(train_T.shape, train_y.shape)

torch.Size([100, 2]) torch.Size([100, 5])


In [31]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_T, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_T, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_T, train_y, likelihood)

In [32]:
# Find optimal model hyperparameters
training_iter = 100

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters 
#Should I change lr? How do I know what would be best?

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_T)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
#     print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#         i + 1, training_iter, loss.item(),
#         model.covar_module.base_kernel.lengthscale.item(),
#          model.likelihood.noise.item()
#     ))
    optimizer.step()

RuntimeError: The size of tensor a (5) must match the size of tensor b (100) at non-singleton dimension 1

In [ ]:
# How do I print the hyperparameters?


In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()
test_Theta1 =  np.linspace(-2,2,50)
test_Theta2 =  np.linspace(-2,2,50)

# Test points are regularly spaced along [-2,2]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_T = torch.tensor([test_Theta1,test_Theta2]).reshape((50,2))
    observed_pred = likelihood(model(test_T))

In [ ]:
# with torch.no_grad():
# Initialize plot
f, ax = plt.subplots()

# Get upper and lower confidence bounds
lower, upper = observed_pred.confidence_region()
# Plot training data as black stars
ax.plot(train_T[0:].reshape(2,100)[0].numpy(), train_y.numpy(), 'k*')
# Plot predictive means as blue line
ax.plot(test_T.reshape(2,50)[0].numpy(), observed_pred.mean.numpy(), 'b') #Need to change
# Shade between the lower and upper confidence bounds
ax.fill_between(test_T.reshape(2,50)[0].numpy(), lower.numpy(), upper.numpy(), alpha=0.5) #Need to change

ax.legend(['Observed Data', 'Mean', 'Confidence'])

In [ ]:
# with torch.no_grad():
# Initialize plot
f, ax = plt.subplots()

# Get upper and lower confidence bounds
lower, upper = observed_pred.confidence_region()
# Plot training data as black stars
ax.plot(train_T[0:].reshape(2,100)[1].numpy(), train_y.numpy(), 'k*')
# Plot predictive means as blue line
ax.plot(test_T.reshape(2,50)[0:][1].numpy(), observed_pred.mean.numpy(), 'b') #Need to change
# Shade between the lower and upper confidence bounds
ax.fill_between(test_T.reshape(2,50)[0:][1].numpy(), lower.numpy(), upper.numpy(), alpha=0.5) #Need to change
ax.legend(['Observed Data', 'Mean', 'Confidence'])